In [6]:
years = list(range(1991,2022))

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time

In [8]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    with open("mvp/{}.html".format(year), "w+") as f:
        f.write(data.text)


In [20]:
dfs =[]
for year in years :

    with open("mvp/{}.html".format(year)) as f :
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr',class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    mvp["year"] = year
    dfs.append(mvp)

In [21]:
mvps = pd.concat(dfs)

In [23]:
mvps.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,year
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,2,Magic Johnson,31,LAL,10.0,497.0,960,0.518,79,37.1,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,3,David Robinson,25,SAS,6.0,476.0,960,0.496,82,37.7,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
3,4,Charles Barkley,27,PHI,2.0,222.0,960,0.231,67,37.3,...,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1991
4,5,Karl Malone,27,UTA,0.0,142.0,960,0.148,82,40.3,...,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1991


In [24]:
mvps.to_csv("mvps.csv")

In [33]:
driver = webdriver.Chrome(executable_path="/Users/21266/chromedriver")

<ipython-input-33-d7abe410196e>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/21266/chromedriver")


In [37]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

for year in years :
    url = player_stats_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)
    html = driver.page_source

    with open("player/{}.html".format(year), "w+") as f :
        f.write(html) 

In [40]:
df = []
for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr',class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")
    player = pd.read_html(str(player_table))[0]
    player["year"] = year
    df.append(player)


In [41]:
players = pd.concat(df)

In [45]:
players.to_csv("players.csv")

In [48]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"
for year in years :
    url = team_stats_url.format(year)
    data = requests.get(url)

    with open("team/{}.html".format(year), "w+") as f:
        f.write(data.text)


In [52]:
dfs = []
for year in years:
    with open("team/{}.html".format(year)) as f :
        page = f.read()

    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr',class_="thead").decompose()
    team_table = soup.find_all(id="divs_standings_E")
    team = pd.read_html(str(team_table))[0]
    team["year"] = year
    team["Team"] = team["Eastern Conference"]
    del team["Eastern Conference"]
    dfs.append(team)

    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr',class_="thead").decompose()
    team_table = soup.find_all(id="divs_standings_W")
    team = pd.read_html(str(team_table))[0]
    team["year"] = year
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    dfs.append(team)

In [53]:
teams = pd.concat(dfs)

In [54]:
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
13,42,30,.583,—,112.4,110.2,2.26,2021,Dallas Mavericks*
14,38,34,.528,4.0,113.3,112.3,1.07,2021,Memphis Grizzlies*
15,33,39,.458,9.0,111.1,112.8,-1.58,2021,San Antonio Spurs
16,31,41,.431,11.0,114.6,114.9,-0.20,2021,New Orleans Pelicans


In [55]:
teams.to_csv("teams.csv")